In [1]:
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM #, AutoTokenizer, AutoModelForCausalLM
import os
import re
import sys
from typing import List
import json

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)

import torch
from datasets import load_dataset
import pandas as pd
 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams

# Update this path to wherever tensorboard is installed in your environment
os.environ['TENSORBOARD_BINARY'] = '/opt/conda/envs/py310/bin/tensorboard'


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /opt/conda did not contain libcudart.so as expected! Searching further paths...
  warn(msg)


In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [5]:
BASE_MODEL = "decapoda-research/llama-13b-hf"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "kmnis/ZenAI")

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [6]:
CUTOFF_LEN = 512

In [7]:
jsons = []
with open("../data/psychology_data.jsonl", "r") as f:
    for line in f:
        jsons.append(json.loads(line))

In [8]:
def split_text_into_chunks(text):
    # Split the text into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # Check if adding the current sentence exceeds the maximum chunk length
        if len(current_chunk) + len(sentence) > CUTOFF_LEN:
            chunks.append(current_chunk.strip())  # Add the current chunk to the list of chunks
            current_chunk = ""  # Reset the current chunk
        
        current_chunk += " " + sentence + " "  # Add the sentence to the current chunk
    
    if current_chunk:
        chunks.append(current_chunk.strip())  # Add the remaining chunk to the list of chunks
    
    return chunks

prompts = []
for book in jsons:
    splitted_text = split_text_into_chunks(book['text'].strip())
    if len(splitted_text):
        for i, c in enumerate(splitted_text):
            prompts.append({
                'id': f"text_id{book['text_id']}_chunk_id{i}",
                'text': c
            })

with open("../data/psychology_prompts.json", "w") as f:
    json.dump(prompts, f)

In [9]:
data = load_dataset("json", data_files="../data/psychology_prompts.json")
data["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jupyter/.cache/huggingface/datasets/json/default-e21eaad9f9272153/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'text'],
    num_rows: 47109
})

In [10]:
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
 
    result["labels"] = result["input_ids"].copy()
 
    return result
 
def generate_and_tokenize_prompt(text):
    return tokenize(text['text'])

In [11]:
train_val = data["train"].train_test_split(
    test_size=500, shuffle=True, seed=42
)
train_data = (
    train_val["train"].map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].map(generate_and_tokenize_prompt)
)

Map:   0%|          | 0/46609 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [12]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

BATCH_SIZE = 256
MICRO_BATCH_SIZE = 8
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
EPOCHS = 20
OUTPUT_DIR = "experiments3"  # Path to where your training model will be saved

In [13]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 6553600 || all params: 13022417920 || trainable%: 0.05032552357220002


In [14]:
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=50,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=5,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=2,
    save_steps=2,
    output_dir=OUTPUT_DIR,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="tensorboard"
)

In [15]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [16]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model.config.use_cache = False
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))
 
model = torch.compile(model)

In [17]:
trainer.train()

/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
2,No log,2.999619
4,No log,2.998737
6,2.986300,2.995750
8,2.986300,2.989688
10,3.006100,2.981690
12,3.006100,2.968946
14,3.006100,2.946289
16,2.948000,2.917622
18,2.948000,2.880727
20,2.883200,2.839146


/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/tmp/ipykernel_16692/4032920361.py:1 in <module>                                             │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/var/tmp/ipykernel_16692/4032920361.py'                    │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/transformers/trainer.py:1664 in train         │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/transformers/trainer.py:1940 in               │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1937 │   │   │   │   │   with model.no_sync():                                                 │
│   1938 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1939 │   │   │   │   else:                                                                     │
│ ❱ 1940 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1941 │   │   │   │                                                                             │
│   1942 │   │   │   │   if (                                                                      │
│   1943 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/transformers/trainer.py:2745 in training_step │
│                                                                                                  │
│   2742 │   │   │   loss = loss / self.args.gradient_accumulation_steps                           │
│   2743 │   │                                                                                     │
│   2744 │   │   if self.do_grad_scaling:                                                          │
│ ❱ 2745 │   │   │   self.scaler.scale(loss).backward()                                            │
│   2746 │   │   elif self.use_apex:                                                               │
│   2747 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2748 │   │   │   │   scaled_loss.backward()                                                    │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/torch/_tensor.py:487 in backward              │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                      

In [18]:
model.save_pretrained(OUTPUT_DIR)

In [19]:
from huggingface_hub import notebook_login
 
notebook_login()

In [20]:
model.push_to_hub("kmnis/ZenAI", use_auth_token=True, private=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/443 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kmnis/ZenAI/commit/625d24c8a3881900b101d6fd952e318da1ec8834', commit_message='Upload model', commit_description='', oid='625d24c8a3881900b101d6fd952e318da1ec8834', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
/opt/conda/envs/py310/lib/python3.10/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.29' not found (required by /opt/conda/envs/py310/lib/python3.10/site-packages/tensorboard_data_server/bin/server)
/opt/conda/envs/py310/lib/python3.10/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /opt/conda/envs/py310/lib/python3.10/site-packages/tensorboard_data_server/bin/server)
/opt/conda/envs/py310/lib/python3.10/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /opt/conda/envs/py310/lib/python3.10/site-packages/tensorboard_data_server/bin/server)
/opt/conda/envs/py310/lib/python3.10/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux